# Генератор ошибок для дальнейшего пробинга

Сделано:
* Замена в случайных словах буквы на близкий символ клавиатуры: `random_keyboard_sub`
* Случайные опечатки (любая буква кириллицы): `random_cyrillic_sub`
* Если надо заменить несколько символов строго подряд: `random_several`

* Псевдолингвистические фичи:

* Проверка


In [ ]:
!pip install pymorphy3 razdel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.1 MB/s eta 0:00:00


In [ ]:
import re
import random
import spacy
import pandas as pd
from pymorphy3 import MorphAnalyzer
from tqdm import tqdm
from collections import Counter
import random
import requests
from razdel import tokenize
from sklearn.model_selection import train_test_split

# Проверка существования слова

## Создание словаря датасета

In [ ]:
sharing_url = "https://drive.google.com/file/d/1bYImwMnmKGTq-QNRrYeI8iHJIlQdSLdz/view?usp=sharing"
download_url = "https://drive.google.com/uc?id=" + sharing_url.split("/")[-2]

test_dataset = pd.read_csv(download_url)
orig_text = ' '.join(test_dataset.base).split()
len(orig_text)

3021712

In [ ]:
# формы оригинального датасета

morphology = MorphAnalyzer()
lemma_cache = dict()

for word in orig_text:
    if word not in lemma_cache.keys():
        lemma = morphology.parse(word)[0].normal_form
        lemma_cache[word] = lemma

dataset_normal_words = set(list(lemma_cache.keys()) + list(lemma_cache.values()))

KeyboardInterrupt: 

In [ ]:
# формы полиперсональных глаголов

polypers_text = ' '.join(test_dataset.polypers).split()
polypers_forms = list()

for word in polypers_text:
    if word not in lemma_cache.keys():
        polypers_forms.append(word)

In [ ]:
dataset_normal_words.update(polypers_forms)

with open('normal_words.txt', 'w') as f:
    for word in dataset_normal_words:
        f.write(word)
        f.write('\n')

## Словарь из википедии

In [ ]:
import requests

url = "https://drive.google.com/file/d/1vgocHX5ldtI3iSjg0Q9aHvf_m7N6zwM4/view?usp=sharing"
download_url = "https://drive.google.com/uc?id=" + url.split("/")[-2]
response = requests.get(download_url)
content = response.text
content

In [ ]:
with open('normal_words.txt', 'r') as f:
    dataset_normal_words = set(word.strip() for word in f)

wiki = open('wiki_data.txt', encoding='utf8').read()
wiki_vocab = set(re.findall('\w+', wiki.lower()))

dataset_normal_words.update(wiki_vocab)

In [ ]:
# консервируем все в файл
with open('normal_words.txt', 'w') as f:
    for word in dataset_normal_words:
        f.write(word)
        f.write('\n')

## Функция для проверки существования слова

In [ ]:
sharing_url = "https://drive.google.com/file/d/1bYImwMnmKGTq-QNRrYeI8iHJIlQdSLdz/view?usp=sharing"
download_url = "https://drive.google.com/uc?id=" + sharing_url.split("/")[-2]

test_dataset = pd.read_csv(download_url)

In [ ]:
with open('normal_words.txt') as f:
    dataset_normal_words = set(word.strip() for word in f)

In [ ]:
def if_regular_word(word):
    if word in dataset_normal_words:
        return True
    elif morphology.parse(word)[0].normal_form in dataset_normal_words:
        return True
    else:
        return False

# Случайные опечатки и клавиатура

## Глобальные переменные

In [ ]:
cyrillic_chars = [chr(c) for c in range(ord('А'), ord('я') + 1)]
cyrillic_chars.extend(['ё', 'Ё', ' '])
morphology = MorphAnalyzer()

In [ ]:
# словарь для соседей по клавиатуре
keyboard_neighbors_low = {
    'й': ['ц', 'ф'],
    'ц': ['й', 'у', 'ф', 'ы'],
    'у': ['ц', 'к', 'ы', 'в'],
    'к': ['у', 'е', 'в', 'а'],
    'е': ['к', 'н', 'а', 'п'],
    'н': ['е', 'г', 'п', 'р'],
    'г': ['н', 'ш', 'р', 'о'],
    'ш': ['г', 'щ', 'о', 'л'],
    'щ': ['ш', 'з', 'л', 'д'],
    'з': ['щ', 'х', 'д', 'ж'],
    'х': ['з', 'ъ', 'ж', 'э'],
    'ъ': ['х', 'э'],

    'ф': ['й', 'ц', 'ы', 'я'],
    'ы': ['ф', 'ц', 'у', 'в', 'я', 'ч'],
    'в': ['ы', 'у', 'к', 'а', 'ч', 'с'],
    'а': ['в', 'к', 'е', 'п', 'с', 'м'],
    'п': ['а', 'е', 'н', 'р', 'м', 'и'],
    'р': ['п', 'н', 'г', 'о', 'и', 'т'],
    'о': ['р', 'г', 'ш', 'л', 'т', 'ь'],
    'л': ['о', 'ш', 'щ', 'д', 'ь', 'б'],
    'д': ['л', 'щ', 'з', 'ж', 'б', 'ю'],
    'ж': ['д', 'з', 'х', 'э', 'ю', '.'],
    'э': ['ж', 'х', 'ъ', '.', 'ю'],

    'я': ['ф', 'ы', 'ч'],
    'ч': ['я', 'ы', 'в', 'с'],
    'с': ['ч', 'в', 'а', 'м'],
    'м': ['с', 'а', 'п', 'и'],
    'и': ['м', 'п', 'р', 'т'],
    'т': ['и', 'р', 'о', 'ь'],
    'ь': ['т', 'о', 'л', 'б'],
    'б': ['ь', 'л', 'д', 'ю'],
    'ю': ['б', 'д', 'ж', 'э'],

    'ё': ['й'],

    ' ': ['с', 'м', 'и', 'т', 'ь'],
    ',': ['ю'],
    '.': ['ю'],
    '!': ['ё', 'й'],
    ':': ['е', 'н'],
    ';': ['у', 'к'],
    '?': ['н', 'г']}

keyboard_neighbors_up = {letter.upper(): [var.upper() for var in vars] for letter, vars in keyboard_neighbors_low.items()}

In [ ]:
class IndexNotInFunc(Exception):
    def __init__(self, message='У вас отключен рандомный ввод, при этом вы не ввели индекс.'):
        self.message = message
        super().__init__(message)

    def __str__(self):
        return f"{self.message}"

## Функции

In [ ]:
def random_cyrillic_sub(word, sub_num=1, randomized=True, index=None, return_index=False):

    mutated_text = word
    while if_regular_word(mutated_text) == True:
        # print(mutated_text)

        mutated_text = word
        cyrillic_indices = [i for i, char in enumerate(word) if char in cyrillic_chars]

        for i in range(sub_num):
            if randomized == True:
                random_index = random.choice(cyrillic_indices)
                while word[random_index] not in cyrillic_chars:
                    random_index = random.choice(cyrillic_indices)
            else:
                if index != None:
                    random_index=index
                else:
                    raise IndexNotInFunc

            original_char = word[random_index]
            if original_char.lower() == original_char:
                possible_chars = [c for c in cyrillic_chars if c != original_char and c.lower() == c]
            else:
                possible_chars = [c for c in cyrillic_chars if c != original_char and c.upper() == c]

            new_char = random.choice(possible_chars)

            mutated_text = mutated_text[:random_index] + new_char + mutated_text[random_index + 1:]

    if return_index == False:
        return mutated_text
    else:
        return mutated_text, random_index

In [ ]:
random_cyrillic_sub('лиса')

'лисц'

In [ ]:
def random_keyboard_sub(word, sub_num=1, randomized=True, index=None, return_index=False):

    mutated_text = word
    while if_regular_word(mutated_text) == True:
        print(mutated_text)

        mutated_text = word
        cyrillic_indices = [i for i, char in enumerate(word) if char in cyrillic_chars]

        for i in range(sub_num):
            if randomized == True:
                random_index = random.choice(cyrillic_indices)
                while word[random_index] not in cyrillic_chars:
                    random_index = random.choice(cyrillic_indices)
            else:
                if index != None:
                    random_index=index
                else:
                    raise IndexNotInFunc

            original_char = word[random_index]
            if original_char in keyboard_neighbors_low.keys():
                possible_chars = keyboard_neighbors_low[original_char]
            elif original_char in keyboard_neighbors_up.keys():
                possible_chars = keyboard_neighbors_up[original_char]

            new_char = random.choice(possible_chars)

            mutated_text = mutated_text[:random_index] + new_char + mutated_text[random_index + 1:]

    if return_index == False:
        return mutated_text
    else:
        return mutated_text, random_index

# Псевдолингвистические фичи

## Глобальные переменные

In [ ]:
noun_endings = list(set([
    # Singular (hard stem)
    "й", "ь",    # masc nom
    "а",             # masc gen
    "у",             # masc dat
    "а",         # masc acc (Nom/Gen)
    "ом",            # masc inst
    "е",             # masc prep

    "а",             # fem nom
    "ы",             # fem gen
    "е",             # fem dat
    "у",             # fem acc
    "ой",            # fem inst
    "е",             # fem prep

    "о",             # neut nom
    "а",             # neut gen
    "у",             # neut dat
    "о",             # neut acc
    "ом",            # neut inst
    "е",             # neut prep

    # Singular (soft stem)
    "ь", "й",        # masc nom
    "я",             # masc gen
    "ю",             # masc dat
    "ь", "я",        # masc acc (Nom/Gen)
    "ем",            # masc inst
    "е",             # masc prep

    "я",             # fem nom
    "и",             # fem gen
    "е",             # fem dat
    "ю",             # fem acc
    "ей", "ёй",      # fem inst
    "е",             # fem prep

    "е",             # neut nom
    "я",             # neut gen
    "ю",             # neut dat
    "е",             # neut acc
    "ем",            # neut inst
    "е",             # neut prep

    # Plural (hard stem)
    "ы", "и",        # nom
    "ов", "ей",      # gen
    "ам",            # dat
    "ы", "и", "ов", "ей", # acc (Nom/Gen)
    "ами",           # inst
    "ах",            # prep

    # Plural (soft stem)
    "и",             # nom
    "ей",            # gen
    "ям",            # dat
    "и", "ей",       # acc (Nom/Gen)
    "ями",           # inst
    "ях"             # prep
]))

adj_endings = list(set([
    # Full forms
    "ый", "ий",      # masc nom
    "ая", "яя",      # fem nom
    "ое", "ее",      # neut nom
    "ого", "его",    # masc/gen/neut gen
    "ой", "ей",      # fem gen/dat/inst/prep
    "ому", "ему",    # masc/neut dat
    "ую", "юю",      # fem acc
    "ым", "им",      # masc/neut inst
    "ом", "ем",      # masc/neut prep

    # Short forms
    "а", "о", "ы",  # masc, fem, neut, pl

    # Comparison
    "ее", "ей",      # comparative
    "ейший", "айший" # superlative
]))

verb_endings = list(set([
    # Present/Future
    "у", "ю",        # 1sg
    "ешь", "ишь",    # 2sg
    "ет", "ит",      # 3sg
    "ем", "им",      # 1pl
    "ете", "ите",    # 2pl
    "ут", "ют", "ат", "ят", # 3pl

    # Past
    "л", "ла", "ло", "ли",

    # Imperative
    "и", "ь",        # sg
    "ите", "ьте",    # pl

    # Infinitive
    "ть", "ти", "чь",
]))

all_endings = list(set(noun_endings + adj_endings + verb_endings))

## Функции

In [ ]:
# функция для преобразования предложения
def sent_modify(sent, sub_num=1):
    tokens = [token.text for token in list(tokenize(sent))]
    punct_marks = [token for token in tokens if re.search(r'[^А-Яа-я]', token) != None]
    # print(punct_marks)
    sent_indices = list(range(len(tokens)))
    modified_idx = list()
    allowed_pos = ['NOUN', 'ADJF', 'PRTF', 'VERB', 'INFN']

    while sub_num+1 > len(tokens)-len(punct_marks):
        sub_num -= 1
    # print(punct_marks, sub_num)

    for i in range(sub_num+1):
        random_idx = random.choice(sent_indices)

        pos = str(morphology.parse(tokens[random_idx])[0].tag)
        pos = pos[:pos.find(',')]

        while if_regular_word(tokens[random_idx]) != True \
        or random_idx in modified_idx or \
        re.search(r'[^А-Яа-я]', tokens[random_idx]) != None:
            random_idx = random.choice(sent_indices)

        pos = str(morphology.parse(tokens[random_idx])[0].tag)
        pos = pos[:pos.find(',')]
        mutated_word = tokens[random_idx]

        while if_regular_word(mutated_word) == True:

            if pos == 'NOUN':
                mutated_word += random.choice(noun_endings)

            elif pos == 'ADJF' or pos == 'PRTF':
                mutated_word += random.choice(adj_endings)

            elif pos == 'VERB' or pos == 'INFN':
                mutated_word += random.choice(verb_endings)

            else:
                mutated_word += random.choice(all_endings)

        tokens[random_idx] = mutated_word
        modified_idx.append(random_idx)

    return ' '.join(tokens)

In [ ]:
sent_modify('Да!')

'Даиы !'

# Подготовка датасета

In [ ]:
sum(test_dataset.was_changed)

59801

In [ ]:
# полиперсональные индексы
random.seed(30)
random_state = 30
polypers_rows = test_dataset[test_dataset.base != test_dataset.polypers].copy()

rest_rows = test_dataset[test_dataset.base == test_dataset.polypers]

# другие предложения, которые мы не будем трогать
# normal_idx = random.sample(rest_idx, len(test_dataset)//2-len(polypers_rows))
# normal_rows = test_dataset.iloc[normal_idx].copy()

# normal_set = pd.concat([polypers_rows, normal_rows])
# normal_set['normal'] = True
rest_rows

,Unnamed: 0,base,polypers,was_changed
0,35,Человек шел ночью и спал днем .,Человек шел ночью и спал днем .,False
1,65,"Уже почти стемнело , он пошел дальше , неровно...","Уже почти стемнело , он пошел дальше , неровно...",False
2,70,"— О-о , какой долгий путь тебе пришлось пройти...","— О-о , какой долгий путь тебе пришлось пройти...",False
5,84,Они заговорили очень тихо .,Они заговорили очень тихо .,False
6,146,Потом все повернулись к гостю .,Потом все повернулись к гостю .,False
...,...,...,...,...
229844,2298442,На широкой веранде санатория находились четвер...,На широкой веранде санатория находились четвер...,False
229845,2298446,Целью нашего приезда был санаторий-лечебница д...,Целью нашего приезда был санаторий-лечебница д...,False
229846,2298454,"Как рад ! - воскликнул он , обнимая старого пр...","Как рад ! - воскликнул он , обнимая старого пр...",False
229847,2298469,- Знает ли Ярослав Васильевич о решении строит...,- Знает ли Ярослав Васильевич о решении строит...,False


In [ ]:
normal_rows, pre_distorted_set = train_test_split(rest_rows,
                                                 train_size=len(test_dataset)//2-len(polypers_rows),
                                                 random_state=random_state,
                                                 shuffle=True)

In [ ]:
normal_set = pd.concat([polypers_rows, normal_rows])
pre_distorted_set['normal'] = False
normal_set['normal'] = True
pre_distorted_set['distortion'] = ''
normal_set['distortion'] = normal_set.base

In [ ]:
len(normal_set) + 1 == len(pre_distorted_set)

True

In [ ]:
distorted_set = pre_distorted_set.copy()

def modify_row(row):
    tokens_length = len(row.base.split())
    if tokens_length < 10:
        return sent_modify(row.base)
    elif 10 <= tokens_length <= 20:
        return sent_modify(row.base, sub_num=2)
    else:
        random_sub = random.choice([1, 2, 3])
        return sent_modify(row.base, sub_num=random_sub)

distorted_set['distortion'] = distorted_set.apply(modify_row, axis=1)

In [ ]:
distorted_set[distorted_set['distortion'] == distorted_set['base']]

,Unnamed: 0,base,polypers,was_changed,normal,distortion
83029,830302,-,-,False,False,-
138959,1392367,— Да-да …,— Да-да …,False,False,— Да-да …
128164,1282504,— О-о !,— О-о !,False,False,— О-о !
107834,1078326,— А-а …,— А-а …,False,False,— А-а …
194913,1950763,Чёрт !,Чёрт !,False,False,Чёрт !
...,...,...,...,...,...,...
177034,1772044,Туда-обратно .,Туда-обратно .,False,False,Туда-обратно .
129286,1293916,Га-ля .,Га-ля .,False,False,Га-ля .
4743,47230,[ 4 ] .,[ 4 ] .,False,False,[ 4 ] .
124892,1249917,— Вот-вот …,— Вот-вот …,False,False,— Вот-вот …


In [ ]:
probing_set = pd.concat([normal_set, distorted_set])
probing_set.to_csv('probing_set.csv')

In [ ]:
len(probing_set)

In [ ]:
# distorted_set = pre_distorted_set.copy()
# for i in tqdm(range(len(distorted_set)), desc="Processing sentences"):
#     # print(i)
#     tokens_length = len(distorted_set.iloc[i].base.split())
#     if tokens_length < 10:
#         distorted_set.loc[i].distortion = sent_modify(distorted_set.iloc[i].base)
#     elif 10 <= tokens_length <= 20:
#         distorted_set.loc[i].distortion = sent_modify(distorted_set.iloc[i].base,
#                                                        sub_num=2)
#     else:
#         random_sub = random.choice([1, 2, 3])
#         distorted_set.loc[i].distortion = sent_modify(distorted_set.iloc[i].base,
#                                                  sub_num=random_sub)

In [ ]:
distorted_set